# Machine Learning with PyTorch

This notebook is used to understand how to implement SGD with PyTorch before experimenting with other algorithms.

In [1]:
# test if pytorch is working
import torch
torch.__version__

'2.7.0.dev20250128+cpu'